In [1]:
import os

from tqdm import tqdm
import numpy as np
import pandas as pd

import gensim

In [2]:
tweetDS=pd.read_feather(os.path.join("data", "tweetDS_token.feather"))[['queryday', 'username', 'author_followers', 'author_description', 'retweets', 'replies', 'likes', 'Token', 'textToken']]
tweetDS.head()

,queryday,username,author_followers,author_description,retweets,replies,likes,Token,textToken
0,2022-02-22,silverandsuch,283,None of us individually is more important than...,0,0,0,"[former, guy, traitor, pure, simple, trump, ca...",former guy traitor pure simple trump call puti...
1,2022-02-22,DRDLAW1,1023,"Proud Gemini, that was conceived from two Gemi...",0,0,1,"[ukraine, counter, attack, russia, would, good...",ukraine counter attack russia would good appro...
2,2022-02-22,FOXXXXFIRE5,13672,"#WeStandWithUkraine 🇺🇸🙏🇺🇦 🌻Democrat, Wa. State...",0,0,0,"[blinken, call, meet, russia, 's, lavrov]",blinken call meet russia 's lavrov
3,2022-02-22,MacLinx,10813,"""And so, my fellow Americans: don't ask what y...",1,0,2,"[blinken, announce, would, not, meet, plan, ru...",blinken announce would not meet plan russian c...
4,2022-02-22,Johnson01924774,6,,0,0,0,"[putin, invade, ukraine, day, factionwar, begi...",putin invade ukraine day factionwar begin anyo...


In [3]:
tweetDS["author_followers"]=tweetDS.author_followers.astype('int32')
tweetDS["likes"]=tweetDS.likes.astype('int32')

In [4]:
tweetDS["author_followers"].quantile([0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1])

0.0           0.0
0.1           8.0
0.2          27.0
0.3          65.0
0.4         133.0
0.5         255.0
0.6         475.0
0.7         910.0
0.8        1968.0
0.9        5725.0
1.0    62498980.0
Name: author_followers, dtype: float64

Let's see if there is some difference between top and down authors

In [5]:
dict_top = gensim.corpora.Dictionary(list(tweetDS.Token[tweetDS.author_followers>=5725]))

In [6]:
bow_corpus_top = [dict_top.doc2bow(doc) for doc in list(tweetDS.Token[tweetDS.author_followers>=5725])]
bow_corpus_top

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(1, 2),
  (3, 1),
  (4, 2),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 2),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 2),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1)],
 [(5, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)],
 [(19, 1),
  (23, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1)],
 [(23, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1)],
 [(7, 1),
  (18, 1),
  (23, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1)],
 [(5, 1),
  (23, 1),
  (39, 1),
  (47, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1)],
 [(5, 1),
  (23, 1),

In [19]:
lda_model_top =  gensim.models.LdaMulticore(bow_corpus_top, 
                                   num_topics = 3, 
                                   id2word = dict_top,
                                   passes = 5,
                                   workers = 3)

In [20]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_top.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.041*"ukraine" + 0.033*"not" + 0.027*"russia" + 0.011*"putin" + 0.011*"be" + 0.010*"people" + 0.007*"amp" + 0.007*"war" + 0.006*"go" + 0.006*"would"


Topic: 1 
Words: 0.039*"ukraine" + 0.037*"russia" + 0.016*"war" + 0.011*"'s" + 0.009*"putin" + 0.008*"say" + 0.008*"russian" + 0.005*"invasion" + 0.005*"biden" + 0.004*"world"


Topic: 2 
Words: 0.054*"ukraine" + 0.025*"russia" + 0.022*"russian" + 0.015*"putin" + 0.013*"invasion" + 0.013*"'s" + 0.009*"president" + 0.009*"say" + 0.009*"sanction" + 0.008*"biden"




In [9]:
dict_down = gensim.corpora.Dictionary(list(tweetDS.Token[tweetDS.author_followers<=8]))

In [10]:
bow_corpus_down = [dict_down.doc2bow(doc) for doc in list(tweetDS.Token[tweetDS.author_followers<=8])]
bow_corpus_down

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1)],
 [(11, 1),
  (12, 1),
  (13, 1),
  (14, 2),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 2),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1)],
 [(32, 1), (38, 1), (39, 1), (40, 1), (41, 1)],
 [(10, 1), (32, 1), (42, 1), (43, 1), (44, 1), (45, 1)],
 [(7, 1),
  (32, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1)],
 [(10, 1),
  (27, 1),
  (32, 1),
  (45, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 2),
  (67, 1)],
 [(10, 1),
  (66, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1),


In [21]:
lda_model_down =  gensim.models.LdaMulticore(bow_corpus_down, 
                                   num_topics = 3, 
                                   id2word = dict_down,
                                   passes = 5,
                                   workers = 3)

In [22]:
for idx, topic in lda_model_down.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.040*"russia" + 0.031*"not" + 0.027*"ukraine" + 0.015*"putin" + 0.011*"biden" + 0.010*"trump" + 0.009*"be" + 0.008*"war" + 0.008*"'s" + 0.008*"would"


Topic: 1 
Words: 0.055*"russia" + 0.029*"ukraine" + 0.023*"not" + 0.015*"putin" + 0.015*"russian" + 0.015*"nato" + 0.009*"country" + 0.008*"war" + 0.007*"people" + 0.007*"be"


Topic: 2 
Words: 0.082*"ukraine" + 0.020*"not" + 0.015*"people" + 0.012*"help" + 0.011*"war" + 0.010*"russia" + 0.009*"russian" + 0.008*"fight" + 0.008*"stop" + 0.007*"go"




Now we check if different authors use different words

In [27]:
tweetDS[["username", "author_followers"]].groupby('username').mean().sort_values("author_followers", ascending=False).head(60)

,author_followers
username,
cnnbrk,6.249898e+07
CNN,5.692021e+07
nytimes,5.213558e+07
BBCWorld,3.520844e+07
TheEconomist,2.638113e+07
Reuters,2.469069e+07
FoxNews,2.086539e+07
POTUS,1.961756e+07
WSJ,1.957819e+07


In [56]:
from nltk import tokenize

doc=". ".join(tweetDS.textToken[tweetDS.username=="CNN"])

total_words = doc.split()
total_word_length = len(total_words)
print(f"total_word_length: {total_word_length}")

total_sentences = tokenize.sent_tokenize(doc)
total_sent_len = len(total_sentences)
print(f"total_sent_len: {total_sent_len}")

total_word_length: 228
total_sent_len: 12


In [57]:
tf_score = {}
for each_word in total_words:
    each_word = each_word.replace('.','').replace("'s", "")
    if each_word in tf_score:
        tf_score[each_word] += 1
    else:
        tf_score[each_word] = 1
    

# Dividing by total_word_length for each dictionary element
tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
print(tf_score)

{'secretary': 0.0043859649122807015, 'state': 0.0043859649122807015, 'blinken': 0.0043859649122807015, 'call': 0.008771929824561403, 'plan': 0.0043859649122807015, 'meeting': 0.0043859649122807015, 'russia': 0.03070175438596491, '': 0.02631578947368421, 'foreign': 0.0043859649122807015, 'minister': 0.0043859649122807015, 'late': 0.008771929824561403, 'sign': 0.0043859649122807015, 'diplomatic': 0.0043859649122807015, 'avenue': 0.0043859649122807015, 'close': 0.0043859649122807015, 'stock': 0.0043859649122807015, 'slide': 0.0043859649122807015, 'simmer': 0.0043859649122807015, 'ukraine': 0.03508771929824561, 'tension': 0.0043859649122807015, 'rattle': 0.0043859649122807015, 'investor': 0.0043859649122807015, 'dow': 0.0043859649122807015, 'nasdaq': 0.0043859649122807015, 'post': 0.0043859649122807015, 'fifth': 0.0043859649122807015, 'straight': 0.0043859649122807015, 'day': 0.008771929824561403, 'loss': 0.008771929824561403, 'effective': 0.0043859649122807015, 'way': 0.004385964912280701

In [58]:
#This method will be required when calculating IDF
def check_sent(word, sentences): 
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))

In [59]:
import math

idf_score = {}
for each_word in total_words:
    each_word = each_word.replace('.','').replace("'s", "")
    if each_word in idf_score:
        idf_score[each_word] = check_sent(each_word, total_sentences)
    else:
        idf_score[each_word] = 1

# Performing a log and divide
idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())

print(idf_score)

{'secretary': 2.4849066497880004, 'state': 2.4849066497880004, 'blinken': 2.4849066497880004, 'call': 0.0, 'plan': 2.4849066497880004, 'meeting': 2.4849066497880004, 'russia': 0.0, '': 0.0, 'foreign': 2.4849066497880004, 'minister': 2.4849066497880004, 'late': 0.0, 'sign': 2.4849066497880004, 'diplomatic': 2.4849066497880004, 'avenue': 2.4849066497880004, 'close': 2.4849066497880004, 'stock': 2.4849066497880004, 'slide': 2.4849066497880004, 'simmer': 2.4849066497880004, 'ukraine': 0.08701137698962969, 'tension': 2.4849066497880004, 'rattle': 2.4849066497880004, 'investor': 2.4849066497880004, 'dow': 2.4849066497880004, 'nasdaq': 2.4849066497880004, 'post': 2.4849066497880004, 'fifth': 2.4849066497880004, 'straight': 2.4849066497880004, 'day': 0.08701137698962969, 'loss': 0.0, 'effective': 2.4849066497880004, 'way': 2.4849066497880004, 'target': 2.4849066497880004, 'sanction': 2.4849066497880004, 'cut': 2.4849066497880004, 'supply': 2.4849066497880004, 'oil': 2.4849066497880004, 'natura

In [60]:
tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
print(tf_idf_score)

{'secretary': 0.010898713376263159, 'state': 0.010898713376263159, 'blinken': 0.010898713376263159, 'call': 0.0, 'plan': 0.010898713376263159, 'meeting': 0.010898713376263159, 'russia': 0.0, '': 0.0, 'foreign': 0.010898713376263159, 'minister': 0.010898713376263159, 'late': 0.0, 'sign': 0.010898713376263159, 'diplomatic': 0.010898713376263159, 'avenue': 0.010898713376263159, 'close': 0.010898713376263159, 'stock': 0.010898713376263159, 'slide': 0.010898713376263159, 'simmer': 0.010898713376263159, 'ukraine': 0.003053030771565954, 'tension': 0.010898713376263159, 'rattle': 0.010898713376263159, 'investor': 0.010898713376263159, 'dow': 0.010898713376263159, 'nasdaq': 0.010898713376263159, 'post': 0.010898713376263159, 'fifth': 0.010898713376263159, 'straight': 0.010898713376263159, 'day': 0.0007632576928914885, 'loss': 0.0, 'effective': 0.010898713376263159, 'way': 0.010898713376263159, 'target': 0.010898713376263159, 'sanction': 0.010898713376263159, 'cut': 0.010898713376263159, 'supply

In [62]:
from operator import itemgetter
def get_top_n(dict_elem, n):
    result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
    return result

print(get_top_n(tf_idf_score, 10))

{'zelensky': 0.014455424850896181, 'secretary': 0.010898713376263159, 'state': 0.010898713376263159, 'blinken': 0.010898713376263159, 'plan': 0.010898713376263159, 'meeting': 0.010898713376263159, 'foreign': 0.010898713376263159, 'minister': 0.010898713376263159, 'sign': 0.010898713376263159, 'diplomatic': 0.010898713376263159}


In [63]:
doc=". ".join(tweetDS.textToken[(tweetDS.username=="BBCWorld")|(tweetDS.username=="BBCNews")])

total_words = doc.split()
total_word_length = len(total_words)
print(f"total_word_length: {total_word_length}")

total_sentences = tokenize.sent_tokenize(doc)
total_sent_len = len(total_sentences)
print(f"total_sent_len: {total_sent_len}")

total_word_length: 119
total_sent_len: 16


In [64]:
tf_score = {}
for each_word in total_words:
    each_word = each_word.replace('.','').replace("'s", "")
    if each_word in tf_score:
        tf_score[each_word] += 1
    else:
        tf_score[each_word] = 1
    

# Dividing by total_word_length for each dictionary element
tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())

idf_score = {}
for each_word in total_words:
    each_word = each_word.replace('.','').replace("'s", "")
    if each_word in idf_score:
        idf_score[each_word] = check_sent(each_word, total_sentences)
    else:
        idf_score[each_word] = 1

# Performing a log and divide
idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())

In [65]:
tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}

In [66]:
print(get_top_n(tf_idf_score, 10))

{'tomorrowspaperstoday': 0.0349485973391569, 'putin': 0.02472678789105192, 'announce': 0.023299064892771268, 'full': 0.023299064892771268, 'block': 0.023299064892771268, 'sanction': 0.023299064892771268, 'wednesday': 0.023299064892771268, 'threat': 0.023299064892771268, 'grow': 0.023299064892771268, 'back': 0.023299064892771268}
